# computer Jacobian for 3 observation (old and new)

In [1]:
from scipy.stats import ortho_group
import numpy as np
from scipy.linalg import expm

def skew_np(vec_):
    vec = np.squeeze(vec_.reshape(3,))
    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    return np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

# Generate a random 3x3 orthogonal matrix
random_orthogonal_matrix = ortho_group.rvs(3)
random_orthogonal_matrix_2 = ortho_group.rvs(3)
v_init = np.array([1, 2, 3]).reshape(3, 1)
p_init = np.array([4, 5, 6]).reshape(3, 1)

np.set_printoptions(precision=4)

In [2]:

def h_acc_all(state, a_n, omega_n, i):
    global jacobian_analytical
    R_imu, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]
    
    # update the values of the state after perturbation

    # TODO : use the closed form (42) in the paper instead
    R_c = expm(skew_np(state[15:18].reshape(3,))) @ random_orthogonal_matrix

    # print("the term 4/7 prior \n", (R_c.T @ skew_np(p_c.reshape(3,))))
    previous_chi = np.vstack((
                            np.hstack((random_orthogonal_matrix_2, v_init, p_init)),
                            np.hstack((np.zeros((2, 3)), np.eye(2)))
                        ))
    
    skew_5_5 = np.vstack((
        np.hstack((skew_np(state[0:3].reshape(3,)), v_imu, p_imu)),
        np.zeros((2, 5))
    ))
    
    R_imu = (expm(skew_5_5) @ previous_chi )[:3, :3]
    b_vector = (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)).reshape(3,)
    
    vee = (omega_n - b_omega).reshape(3,)
    pee = np.copy(p_c)


    M1 = np.array([0, 1, 0]).reshape(1, 3)
    M2 = np.array([1, 0, 0]).reshape(1, 3)
    M3 = np.array([0, 0, 1]).reshape(1, 3)

    omega_p = np.cross(vee, p_c)
    alpha1 = (a_n - b_a + np.cross(vee, omega_p)).reshape(3,)
    alpha2 = (R_imu.T @ v_imu ).flatten() + np.cross(omega_n - b_omega, p_c)
    alpha3 = (vee).reshape(3,1)
    
    a_car = R_c.T @ alpha1
    v_c = R_c.T @ alpha2
    omega_car = R_c.T @ alpha3

    ############## find analytical solution #################

    #  only print one time
    if(i == 20):
        ############## for old observation #################
        term_1_7_old = np.zeros((3, 3))
        term_2_7_old = (R_c.T  @ R_imu.T)
        term_3_7_old = np.zeros((3, 3))
        term_4_7_old = (R_c.T @ skew_np(p_c.reshape(3,)))
        term_5_7_old = np.zeros((3, 3))
        term_6_7_old = (R_c.T @ skew_np (b_vector))
        term_7_7_old = (R_c.T @ skew_np(vee))
        J_old = np.hstack([term_1_7_old, term_2_7_old, term_3_7_old, term_4_7_old, term_5_7_old, term_6_7_old, term_7_7_old])[1:3, :]

        ############## for new observation #################
        term_1_7_new = np.zeros((1, 3))
        term_2_7_new = - M2 @ (R_c.T @ R_imu.T) * (omega_car[2])
        term_3_7_new = np.zeros((1, 3))
        term_4_7_new =  M1 @ R_c.T @ ( - np.outer(vee, pee) - np.squeeze((omega_n - b_omega).dot(p_c)) * np.eye(3) + (2 * np.outer(pee, vee))) \
                        + (1 * M2 @ R_c.T @ skew_np(p_c)) * (-omega_car[2]) \
                        + (-1 * M3 @ R_c.T) * (-v_c[0])
        term_5_7_new = -1 * M1 @ R_c.T 
        term_6_7_new = M1 @ R_c.T @ skew_np(alpha1) \
                   - np.squeeze(np.array([1,0,0]) @ R_c.T @ skew_np(alpha2)) * omega_car[2] \
                   - v_c[0] * (M3 @ R_c.T @ skew_np(alpha3))
        # https://arxiv.org/pdf/1312.0788 equation (12)
        omega_real = omega_n - b_omega
        term_7_7_new = M1 @ R_c.T @ (np.outer(omega_real,omega_real) - (omega_real.T).dot(omega_real) * np.eye(3)) \
                            + M2 @ R_c.T @ skew_np( - omega_real) * (omega_car[2])
        J_new = np.hstack([term_1_7_new, term_2_7_new, term_3_7_new, term_4_7_new, term_5_7_new, term_6_7_new, term_7_7_new])   
        
        jacobian_analytical = np.vstack([J_old, J_new])
        
    obs_new = a_car[1] - v_c[0] * omega_car[2]
    return  np.array([v_c[1].item(), v_c[2].item(), obs_new.item()]).T

def compute_jacobian_all(x_0, a_n, omega_n, d=1e-9):

    # Define small perturbations
    n = 21
    jacobian_in = np.zeros((3, n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc_all(x_0, a_n, omega_n, 10)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc_all(x_0_perturbed, a_n, omega_n, i)
        # Calculate Jacobian element by finite difference
        jacobian_in[:, i] = (obs_perturbed - obs_nominal) / d

    return jacobian_in



In [3]:
# Example usage:
chi_imu = np.array([0.0, 0.0, 0.0])
v_imu = np.array([0.0, 0.0, 0.0])
p_imu = np.array([0.0, 0.0, 0.0])
b_a = np.array([0.0, 0.0, 0.0])
b_omega = np.array([0.5, 0.01, 0.01])
chi_c = np.array([0.0, 0.0, 0.0])
p_c = np.array([0.1, 0.1, 0.1])

# Nominal state

a_n = np.array([1.0, 1.0, 1.0])
omega_n = np.array([0.5, 0.1, 1.0])
x_0 = np.concatenate((chi_imu, v_imu, p_imu, b_omega, b_a, chi_c, p_c))

jacobian_numerical = compute_jacobian_all(x_0, a_n, omega_n)

print("the error is: ", np.linalg.norm(jacobian_numerical - jacobian_analytical))

print(jacobian_numerical.T)
print(jacobian_analytical.T)


the error is:  4.1730970425535805e-07
[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [-0.0183  0.807   0.5746]
 [-0.0378  0.5894 -0.7855]
 [-0.9991 -0.0371  0.0192]
 [ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.0211 -0.0688 -0.2379]
 [-0.0949  0.1047  0.0299]
 [ 0.0738 -0.0358  0.4585]
 [ 0.      0.     -0.972 ]
 [ 0.      0.     -0.2338]
 [ 0.      0.     -0.0231]
 [-0.0044 -0.0973  0.1748]
 [ 0.0067 -0.0871 -0.992 ]
 [ 0.1173  0.0157  0.6148]
 [ 0.2294  0.1816 -1.8854]
 [-0.9623  0.087  -0.4372]
 [ 0.0875 -0.0079  0.0397]]
[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [-0.0183  0.807   0.5746]
 [-0.0378  0.5894 -0.7855]
 [-0.9991 -0.0371  0.0192]
 [ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.0211 -0.0688 -0.2379]
 [-0.0949  0.1047  0.0299]
 [ 0.0738 -0.0358  0.4585]
 [ 0.      0.     -0.972 ]
 [ 0.      0.     -0.2338]
 [ 0.      0.   